In [13]:
from dataclasses import asdict
from pydantic.dataclasses import dataclass
import pydantic.json
import pydantic.tools
from pydantic import BaseModel, Field
import json
from typing import *

In [51]:
# Discriminated unions require a little bit more hand holding
# particularly in the case where you don't want to rely on the default
# strategy adopted by pydantic. This is important when you need to 
# consider multiple languages

# https://docs.pydantic.dev/latest/concepts/unions/#discriminated-unions-with-str-discriminators

@dataclass
class Foo:
    id: int
    name: str
    meepType : Literal['Foo'] = Field(default='Foo')
    
@dataclass
class Bar:
    cost: int
    time: str
    meepType : Literal['Bar'] = Field(default='Bar')
    
Blek = Annotated[Union[Foo, Bar], Field(discriminator='meepType')]

@dataclass
class Baz:
    desc : str
    meep : List[Blek]

In [49]:
o1 = Baz(desc='hello', meep=[ Foo(id = 1, name = 'fizz') ])
o2 = Baz(desc='nope', meep=[ 
    Bar(cost=1, time='hello'), 
    Foo(id = 1, name = 'fizz')
])
olst = [o1, o2]

In [50]:
for o in olst:
    print(o)
    s = json.dumps(o, default=pydantic.json.pydantic_encoder)
    print(s)
    x = pydantic.TypeAdapter(Baz).validate_json(s)
    print(x)

Baz(desc='hello', meep=[Foo(id=1, name='fizz', meepType='Foo')])
{"desc": "hello", "meep": [{"id": 1, "name": "fizz", "meepType": "Foo"}]}
Baz(desc='hello', meep=[Foo(id=1, name='fizz', meepType='Foo')])
Baz(desc='nope', meep=[Bar(cost=1, time='hello', meepType='Bar'), Foo(id=1, name='fizz', meepType='Foo')])
{"desc": "nope", "meep": [{"cost": 1, "time": "hello", "meepType": "Bar"}, {"id": 1, "name": "fizz", "meepType": "Foo"}]}
Baz(desc='nope', meep=[Bar(cost=1, time='hello', meepType='Bar'), Foo(id=1, name='fizz', meepType='Foo')])
